In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from dotenv import load_dotenv
from operator import itemgetter 
import os

from tqdm import tqdm

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
def load_vdb_and_retriever(path="./nlp_cs_theory",
                           k=4):
    embedding_size = 1536
    embedding_model = "text-embedding-3-small"
    embeddings = OpenAIEmbeddings(model=embedding_model, dimensions=embedding_size)
    
    vdb = FAISS.load_local(path, 
                           embeddings, 
                           allow_dangerous_deserialization=True)
    
    retriever = vdb.as_retriever(search_kwargs={"k": k})
    
    return vdb, retriever
    

In [4]:
vdb, retriever = load_vdb_and_retriever()

In [5]:
retriever.invoke("How does PoS Tagging work?")

[Document(page_content="[Music] in this segment we're going to come back to the task of part of speech tagging now armed with hmm and see how they can actually do on real data and kind of compare them to other approaches so to start off with we're going to talk about the version of uh hmm that actually works well for part of speech tagging so what we've talked so far is what I'm going to call a quotee unquote byr hmm where the states here are just single tags now recall that hmm make this marov assumption which is that the next step depends only on uh the current state and not anything in the past and it turns out that this is a little bit too aggressive of an independence assumption instead what we want to do is we want to be able to condition on stuff that's a little bit farther in the past but kind of modifying the actual hmm uh model and and and algorithm is a little bit uh annoying so instead the way we can do this is we can think about changing our state space to actually be a pa

In [6]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.documents import Document
from typing import List, Dict

def format_docs(docs: List[Document]) -> str:
    return "\n".join([x.page_content for x in docs])

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

system_prompt = """
You're an AI assistant that will answer questions about natural language processing and computer science theory.

Besides, here is some extra content about natural language processing and/or computer science theory:

[extra content]
{extra_content}
[End of extra content]

--------------------------------------------
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [7]:
chain = (
    {
        "input": RunnablePassthrough(),
        "extra_content": retriever | RunnableLambda(format_docs)
    }
    | prompt 
    | llm 
    | StrOutputParser()
)

In [8]:
response = chain.invoke("What is PoS Tagging?")

In [9]:
print(response)

Part of Speech (PoS) tagging, also known as PoS annotation or PoS labeling, is a fundamental task in natural language processing (NLP) that involves assigning a part of speech tag to each word in a sentence, indicating the word's syntactic category and grammatical function within the sentence. PoS tags can include categories like nouns, verbs, adjectives, adverbs, pronouns, prepositions, conjunctions, and more.

PoS tagging is essential for various NLP tasks such as text analysis, information extraction, machine translation, and syntactic parsing. It helps in understanding the structure of sentences, disambiguating word meanings, and enabling machines to process and analyze human language more effectively.


In [10]:
chain.invoke("What is Big O notation?")

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 16722 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [14]:
response = chain.invoke({"input": "Explain NP Hardness problems"})

In [15]:
print(response)

NP-hardness refers to a classification of computational problems in complexity theory. A problem is considered NP-hard if it is at least as hard as the hardest problems in the complexity class NP (nondeterministic polynomial time). In other words, if a problem is NP-hard, it means that it is at least as difficult as any problem in NP.

Here are some key points about NP-hardness problems:

1. **Definition**: A problem is NP-hard if every problem in NP can be reduced to it in polynomial time. This means that if you can solve an NP-hard problem in polynomial time, you can solve any problem in NP in polynomial time.

2. **Complexity Class NP**: NP stands for nondeterministic polynomial time. It includes problems for which a proposed solution can be verified in polynomial time but not necessarily found in polynomial time.

3. **NP-Completeness**: Some problems in NP are so hard that they are both NP-hard and in NP. These problems are called NP-complete. If you can solve an NP-complete probl

In [16]:
retriever.invoke("Explain NP Hardness problems")

[Document(page_content="one famous hardest assumption pitas not equal NP but there's more to life than just that so I want to tell you about some more opportunities you have I think we know it's super hard in complexity theory to prove any impossibility results for algorithms I mean we basically cannot do it at all I mean the one thing that we're great at as researchers is developing algorithms and the one thing we're bad at is proving our rules do not exist efficient algorithms for problems you don't exist so that's why we have assumptions so that we can make progress on the subject of negative results so uh let's get into it when you want to prove hardness results for a certain computational problem way it works this is usually a combination of two things you got a hardness assumption and a reduction and reductions are just algorithms that's kind of nice I mean it reduces in a way the task of proving hardness do you know making let's say one or a limited number of assumptions and the

In [17]:
retriever.invoke("What is beam search?")

[Document(page_content="[Music] thank you in this segment we're going to talk about beam search and more generally other decoding strategies for producing language out of language models so the decoding strategies we're going to talk about are going to apply the two types of models that we think about in this class the first is language models uh just placing distributions over the next word and the second are sequence to sequence models placing next word distributions over an output y conditioned on an input X so for in both of these cases at the end of the day we have this model and we give the system some input either a prefix of y's or an X or you know we just wanted to straight up generate a story or generate something for us so how do we actually do this generation so one option is just greedy search so we just take the most likely next word at each step and we kind of repeat this and just crank out a sequence that's one approach uh it's going to be the sort of worst of all possi

In [18]:
response = chain.invoke({"input": "What is beam search?"})

In [19]:
print(response)

Beam search is a decoding algorithm commonly used in natural language processing and machine learning for generating sequences of words, such as in language modeling or machine translation tasks. It is an improvement over the greedy search approach and aims to find a sequence with higher probability by considering multiple candidate sequences simultaneously.

In beam search, at each step of generating a sequence, the algorithm keeps track of a fixed number (beam width) of the most promising candidate sequences based on their probabilities. These candidate sequences are expanded by considering possible next words or tokens, and the top candidates are retained while others are pruned. This process continues until the sequence is complete or a predefined stopping criterion is met.

By maintaining a beam of candidate sequences, beam search explores the search space more effectively than greedy search, which simply selects the most likely next word at each step. Beam search strikes a balanc

In [15]:
async for msg in chain.astream("What is Markov Chains?"):
    print(msg, end="", flush=True)

Markov Chains are mathematical systems that undergo transitions from one state to another in a probabilistic manner. These transitions are based on certain probabilities, and the next state only depends on the current state, not on the sequence of events that preceded it. In other words, Markov Chains exhibit the Markov property, which is the memoryless property where the future state depends only on the present state, not on the sequence of events that led to the present state.

Markov Chains are widely used in various fields such as statistics, physics, computer science, and economics to model random processes and systems. They are essential in understanding and analyzing systems that involve stochastic (random) behavior and have applications in areas like modeling weather patterns, stock market fluctuations, text generation, and more.

In [12]:
final_msg = ""

async for msg in chain.astream("What is Markov Chains?"):
    final_msg += msg

In [13]:
final_msg

'Markov Chains are mathematical systems that undergo transitions from one state to another in a probabilistic manner. These transitions depend only on the current state and not on the sequence of events that preceded it, making them memoryless or "Markovian." In a Markov Chain, the future state of the system is predicted based solely on its current state and the probabilities of transitioning to other states.\n\nMarkov Chains are widely used in various fields such as physics, biology, economics, and computer science. They are fundamental in modeling stochastic processes, analyzing random systems, and predicting future outcomes based on probabilistic transitions between states.'

In [17]:
for msg in chain.stream("What is Markov Chains?"):
    print(msg, end="", flush=True)

Markov Chains are mathematical systems that undergo transitions from one state to another in a probabilistic manner. These transitions depend only on the current state and not on the sequence of events that preceded it, following the Markov property. In essence, the future state of the system is solely determined by its current state.

Markov Chains are widely used in various fields such as statistics, physics, biology, finance, and computer science. They are fundamental in modeling stochastic processes, predicting future events, and analyzing systems with random behavior.